# Anglen_7276_210923B6 - atac_seq
This notebook will create all the necessary files, scripts and folders to pre-process the aforementioned project. Is designed to be used in a jupyter server deployed in a system running SLURM. The majority of the scripts and heavy-lifting processes are wrapped up in sbatch scripts.As an end user, in order to pre-process your samples provided in the spread sheet, you will simply need to *run the entire notebook* (Cell > Run all) and the system should take care of the rest for you.
#### Create necessary folder(s)

In [14]:
%%bash
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/metadata
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/raw_reads
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/processed_raw_reads
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/jsons
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/logs

Save metadata file

In [15]:
%%writefile /data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/metadata/atac_seq_download_metadata.Anglen_7276_210923B6.txt
Sequencing core project	sequencing core library name	Name	Paired-end or single-end	Genome	Library type	Blacklist removal
Anglen_7276_210923B6	Control-1	Control_1	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	Control-2	Control_2	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	Control-3	Control_3	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	CHRIR4uM-1	CHRIR4uM_1	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	CHRIR4uM-2	CHRIR4uM_2	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	CHRIR4uM-3	CHRIR4uM_3	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	YAP-GFP-1	YAP_GFP_1	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	YAP-GFP-2	YAP_GFP_2	PE	hg38	ATAC-seq	True
Anglen_7276_210923B6	YAP-GFP-3	YAP_GFP_3	PE	hg38	ATAC-seq	True


Overwriting /data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/metadata/atac_seq_download_metadata.Anglen_7276_210923B6.txt


#### Download FASTQ from dukeds
Create file to download FASTQ files

In [16]:
%%writefile /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/download_Anglen_7276_210923B6.sh
#!/bin/bash
METADATA=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/metadata/atac_seq_download_metadata.Anglen_7276_210923B6.txt
DATA_HOME=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq
mkdir -p ${DATA_HOME}/raw_reads/

cp -R /gpfs/fs1/data/gersbachlab/tda17/Anglen_7276_210923B6/CHIR.YAP5SAgfp.ATACseq/*fastq.gz /gpfs/fs1/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/raw_reads/Anglen_7276_210923B6/

Overwriting /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/download_Anglen_7276_210923B6.sh


Execute file to download files

In [17]:
%%script --out blocking_job_str bash
sbatch -o /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/logs/Anglen_7276_210923B6_download_fastq_files.out \
 -p all \
 /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/download_Anglen_7276_210923B6.sh

Extract blocking job id

In [18]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Merge lanes of FASTQ files

In [19]:
%%writefile /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/merge_lanes_Anglen_7276_210923B6.sh
#!/bin/bash
#SBATCH --array=0-9%20
ORDER=Anglen_7276_210923B6
RAW_DATA_DIR=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/raw_reads/${ORDER}
PROCESSED_DATA_DIR=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/processed_raw_reads/${ORDER}
METADATA=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/metadata/atac_seq_download_metadata.Anglen_7276_210923B6.txt

mkdir -p ${PROCESSED_DATA_DIR}
cd ${PROCESSED_DATA_DIR}

seq_name_header=$(/bin/grep -Eoi "sequencing.?core.?library.?name" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Sequencing core library name not found in ${METADATA}"
    exit 1
fi

name_header=$(/bin/grep -Poi "\tname\t" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Library Name column not found in ${METADATA}"
    exit 1
fi
name_header=$(echo ${name_header} | cut -f2)

seq_type_header=$(head -1 ${METADATA} | /bin/grep -Poi "paired.?end.?or.?single.?end")
if [[ $? == 1 ]];
then
    echo -e "ERROR: Paired-end or single-end column not found in ${METADATA}"
    exit 1
fi

sample_seq_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
sample_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
seq_type=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_type_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');


for read_pair in R1 R2 UMI;
do
    sample_files=$(/bin/ls ${RAW_DATA_DIR}/${sample_seq_name/ /}_S[0-9]*_L[0-9][0-9][0-9]_${read_pair}_* 2> /dev/null)
    if [[ $? != 0 ]]; # If no samples found with that read_pair, continue
    then
        continue;
    fi
    if [[ ${read_pair} == "R1" || (${seq_type/ /} == "PE" || ${seq_type/ /} == "pe") ]];
    then
        # Merge all lanes
        merged=$(basename $(echo ${sample_files} | awk '{print $1}') | sed -e 's/_L[0-9]\{3\}_/_/')
        cat ${sample_files} > ${merged};

        # Rename samples with our sample Names
        dest_filename=$(basename $(echo ${merged} | awk '{print $1}') | sed -r 's/\_S[0-9]+//; s/\_(R1|R2|UMI)\_/\.\1\./; s/\.[0-9]+\.fastq/\.fastq/')
        mv ${merged} ${dest_filename}

        cleaned_dest_filename=${dest_filename/${sample_seq_name/ /}/${sample_name/ /}}

        if [[ ${seq_type/ /} == "SE" || ${seq_type/ /} == "se" ]];
        then
            cleaned_dest_filename=${cleaned_dest_filename/.R1/}
        fi
        
        mv ${dest_filename} ${cleaned_dest_filename}
    fi
done


Overwriting /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/merge_lanes_Anglen_7276_210923B6.sh


Execute file to merge lanes of FASTQ files

In [20]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -o /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/logs/Anglen_7276_210923B6_merge_fastq_files_%a.out \
 -p all \
 --depend afterok:$1 \
 --array 0-8%20 \
 /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/merge_lanes_Anglen_7276_210923B6.sh

Extract blocking job id

In [21]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create JSON files for CWL pipeline files

In [22]:
%%writefile /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/cwl_json_gen_Anglen_7276_210923B6.sh
#!/bin/bash
ORDER=Anglen_7276_210923B6
PROCESSED_DATA_DIR=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/processed_raw_reads/${ORDER}
METADATA=/data/gersbachlab/tda17/Anglen_7276_210923B6/data/atac_seq/metadata/atac_seq_download_metadata.Anglen_7276_210923B6.txt

python /data/reddylab/software/cwl/GGR-cwl/v1.0/json-generator/run.py \
    -m ${METADATA} \
    -d ${PROCESSED_DATA_DIR} \
    -o /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/jsons \
    -t atac-seq \
    --fastq-gzipped \
    --mem 24000 \
    --nthreads 16 \
    --separate-jsons



Overwriting /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/cwl_json_gen_Anglen_7276_210923B6.sh


Execute file to create JSON files

In [23]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -o /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/logs/Anglen_7276_210923B6_cwl_json_gen.out \
 -p all \
 --depend afterok:$1 \
 /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/cwl_json_gen_Anglen_7276_210923B6.sh

Extract blocking job id

In [24]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create SLURM array master bash file for pe-blacklist-removal samples

In [25]:
%%writefile /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/Anglen_7276_210923B6-pe-blacklist-removal.sh
#!/bin/bash
#SBATCH --job-name=cwl_atac_seq
#SBATCH --output=/data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/logs/Anglen_7276_210923B6-pe-blacklist-removal-%a.out
#SBATCH --mail-user=tda17@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=32G
#SBATCH --cpus-per-task=16


export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/common/shared_conda_envs/ucsc/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/phantompeakqualtools-1.2/:$PATH"
export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/tmpdirs/tmp-Anglen_7276_210923B6-pe-blacklist-removal-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/tmpdirs/tmp-Anglen_7276_210923B6-pe-blacklist-removal-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/Anglen_7276_210923B6-pe-blacklist-removal  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/ATAC-seq_pipeline/pipeline-pe-blacklist-removal.cwl \
    /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/jsons/atac_seq_download_metadata.Anglen_7276_210923B6-pe-blacklist-removal-${SLURM_ARRAY_TASK_ID}.json


# Delete any tmpdir not removed by cwltool
rm -rf /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/tmpdirs/tmp-Anglen_7276_210923B6-pe-blacklist-removal-${SLURM_ARRAY_TASK_ID}-


Overwriting /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/Anglen_7276_210923B6-pe-blacklist-removal.sh


Execute SLURM array master file

In [26]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p all \
 --depend afterok:$1 \
 --array 0-8%20 \
 /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/Anglen_7276_210923B6-pe-blacklist-removal.sh

Extract blocking job id

In [27]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create QC generating script

In [28]:
%%writefile /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/generate_qc_cell_Anglen_7276_210923B6-pe-blacklist-removal.sh
#!/bin/bash
#SBATCH --job-name=qc
#SBATCH --output=/data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/logs/qc_gen.Anglen_7276_210923B6-pe-blacklist-removal.out

source /data/reddylab/software/miniconda2/bin/activate alex
cd /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/Anglen_7276_210923B6-pe-blacklist-removal

python /data/reddylab/software/cwl/bin/generate_stats_atacseq_paired_end.py ./ \
    -samples $(/bin/ls -1 *PBC.txt | sed 's@.PBC.txt@@') \
> qc.txt

Overwriting /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/generate_qc_cell_Anglen_7276_210923B6-pe-blacklist-removal.sh


Generate QCs for Anglen_7276_210923B6-pe-blacklist-removal

In [29]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -p all \
 /data/gersbachlab/tda17/Anglen_7276_210923B6/processing/atac_seq/scripts/generate_qc_cell_Anglen_7276_210923B6-pe-blacklist-removal.sh

Extract blocking job id

In [30]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)